In [ ]:
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds
from brainflow.data_filter import DataFilter, FilterTypes
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

params = BrainFlowInputParams()
params.serial_port = 'COM3'  # Change this depending on your device and OS
board_id = 38  # Change this depending on your device

# Prepares the board for reading data
try:
    board_id = 38
    board = BoardShim(board_id, params)
    board.prepare_session()
    print("Successfully prepared physical board.")
except Exception as e:
    print(e)
    # If the device cannot be found or is being used elsewhere, creates a synthetic board instead
    print("Device could not be found or is being used by another program, creating synthetic board.")
    board_id = BoardIds.SYNTHETIC_BOARD
    board = BoardShim(board_id, params)
    board.prepare_session()

# Releases the board session
board.release_session()

# Start streaming
print("Starting Stream")
board.prepare_session()
board.start_stream()
print("Stream Started")
time.sleep(10)  # wait 10 seconds
data = board.get_board_data()  # gets all data from board and removes it from internal buffer
print("Ending stream")
board.stop_stream()
board.release_session()

# Isolate just the EEG data
eeg_channels = board.get_eeg_channels(board_id)
print(eeg_channels)
eeg_data = data[eeg_channels]
print(eeg_data.shape)

# Function to analyze steady periods and fluctuations
def analyze_brainwave_activity(data, steady_threshold=5.0, fluctuation_threshold=50.0):
    results = []
    for channel_idx in range(data.shape[0]):
        channel_data = data[channel_idx]
        steady_segments = []
        fluctuation_comments = []

        # Analyze steady periods
        start_idx = 0
        for i in range(1, len(channel_data)):
            diff = abs(channel_data[i] - channel_data[i - 1])
            if diff > steady_threshold:
                # End current steady segment
                if i - 1 > start_idx:
                    steady_segments.append((start_idx, i - 1))
                start_idx = i

        # Handle last segment
        if start_idx < len(channel_data) - 1:
            steady_segments.append((start_idx, len(channel_data) - 1))

        # Analyze large fluctuations
        for i in range(1, len(channel_data)):
            diff = abs(channel_data[i] - channel_data[i - 1])
            if diff > fluctuation_threshold:
                fluctuation_comments.append(
                    f"Channel {channel_idx + 1}: Huge fluctuation from {channel_data[i - 1]:.2f} µV to {channel_data[i]:.2f} µV at sample {i}."
                )

        # Store results for this channel
        results.append({
            "channel": channel_idx + 1,
            "steady_segments": steady_segments,
            "fluctuations": fluctuation_comments,
        })

    return results

# Analyze EEG data
analysis_results = analyze_brainwave_activity(eeg_data)

# Output analysis results
for result in analysis_results:
    print(f"\nChannel {result['channel']}:")
    print("Steady segments (start to end sample):")
    for segment in result["steady_segments"]:
        print(f"  - {segment[0]} to {segment[1]}")
    print("Fluctuation comments:")
    for comment in result["fluctuations"]:
        print(f"  - {comment}")

# Save analysis results to CSV
steady_segments_df = pd.DataFrame([
    {"channel": r["channel"], "start": s[0], "end": s[1]}
    for r in analysis_results for s in r["steady_segments"]
])
steady_segments_df.to_csv("steady_segments.csv", index=False)

fluctuation_comments_df = pd.DataFrame([
    {"channel": r["channel"], "comment": c}
    for r in analysis_results for c in r["fluctuations"]
])
fluctuation_comments_df.to_csv("fluctuation_comments.csv", index=False)

# Save EEG data
DataFilter.write_file(eeg_data, 'eeg_data_test.csv', 'w')  # Writes into a CSV file in the current directory

restored_data = DataFilter.read_file('eeg_data_test.csv')  # Reads file back
print(restored_data.shape)

# This shows how much the saved data differs from the original data, they are very similar but not equal.
print(eeg_data - restored_data)

# Visualize EEG data
offsets = [0, 100, 200, 300]
for i in range(eeg_data.shape[0]):
    plt.plot(np.arange(eeg_data.shape[1]), eeg_data[i] + offsets[i], label=f"Stream(i+1)")

plt.xlabel('Time')
plt.ylabel('Amplitude (with offsets)')
plt.legend()
plt.title('EEG Data with Streams Offset')
plt.show()
